# COVID-19 In Washington State
> Tracking coronavirus cases by county, age and gender across Washington

- comments: true
- author: Bilal Tahir
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-wa/

In [20]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
import altair as alt
import gcsfs
from IPython.display import HTML

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [21]:
#hide_input
HTML("<div>Hello World!</div>")

In [28]:
#hide
url = 'gs://covid_wa/dfCasesByCounty.csv'
df = pd.read_csv(url)

df.head(30)

,County,Confirmed,Deaths
0,Benton,7,2
1,Chelan,3,0
2,Clallam,2,0
3,Clark,9,3
4,Columbia,1,0
5,Cowlitz,2,0
6,Franklin,2,0
7,Grant,11,1
8,Grays Harbor,1,0
9,Island,19,1


In [13]:
#hide
dcols = cols[cols.str.match(pat='\d+\/\d+\/\d+', case=False)]
rcols = cols[~cols.isin(dcols)]
df = pd.melt(confirmed_df, 
             id_vars=rcols, value_vars=dcols,
             var_name='date', 
             value_name='confirmed_count')
df.columns = df.columns.str.lower().str.replace('/','_')
df['date'] = pd.to_datetime(df['date'])
df = df.astype({'province_state': 'str','country_region':str})

#clean
df.loc[df.province_state.isna(),'province_state'] = ''

In [14]:
#hide
abbr2state = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

state2abbr = {s:a for a,s in abbr2state.items()}

In [15]:
#hide
us_df = df[(df.country_region == 'US')].copy()

def clean_state(col):
    if ',' in col:
        return col.split(', ', 1)[1]
    else:
        return state2abbr.get(col)

us_df['state'] = us_df.province_state.apply(clean_state)
us_df['state'] = us_df.apply(lambda x: 'DP' if x['province_state'] == 'Diamond Princess' else x['state'], axis=1)

us_df.confirmed_count.fillna(0, inplace=True)
us_df.rename(columns={'country_region':'country'}, 
             inplace=True)

us_daily_df = (us_df
         .groupby(['state','country','date'])
         .agg(
             confirmed_count=('confirmed_count','sum'), 
             lat=('lat','mean'),
             long=('long','mean'))
        )

us_daily_df['new_cases'] = us_daily_df.confirmed_count.diff()
us_daily_df.loc[us_daily_df.new_cases < 0, 'new_cases'] = 0

#us_daily_df['cs'] = us_daily_df.sort_values('date').groupby(['state','country']).confirmed_count.cumsum()

us_daily_df = us_daily_df.reset_index()

#us_daily_df.head()

#active = use the recent date
state_df = us_daily_df.sort_values('date').groupby(['state','country']).tail(1)
#state_df.head()


#https://github.com/altair-viz/altair/issues/1005#issuecomment-403237407
def to_altair_datetime(dt):
    return alt.DateTime(year=dt.year, month=dt.month, date=dt.day,
                        hours=dt.hour, minutes=dt.minute, seconds=dt.second,
                        milliseconds=0.001 * dt.microsecond)

In [41]:
#hide
states_data = 'https://vega.github.io/vega-datasets/data/us-10m.json'
states = alt.topo_feature(states_data, feature='states')
selector = alt.selection_single(empty='none', fields=['state'], nearest=True, init={'state':'WA'})

curr_date = state_df.date.max().date().strftime('%Y-%m-%d')
dmax = (us_daily_df.date.max() + pd.DateOffset(days=3))
dmin = us_daily_df.date.min()

# US states background
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=400
).project('albersUsa')


points = alt.Chart(state_df).mark_circle().encode(
    longitude='long:Q',
    latitude='lat:Q',
    size=alt.Size('confirmed_count:Q', title= 'Number of Confirmed Cases'),
    color=alt.value('steelblue'),
    tooltip=['state:N','confirmed_count:Q']
).properties(
    title=f'Total Confirmed Cases by State as of {curr_date}'
).add_selection(selector)


final_chart = alt.vconcat(
    background + points, 
).resolve_scale(
    color='independent',
    shape='independent',
).configure(
    padding={'left':20, 'bottom':40}
).configure_axis(
    labelFontSize=10,
    labelPadding=10,
    titleFontSize=12,
).configure_view(
     stroke=None
)

In [42]:
#hide_input
final_chart

alt.VConcatChart(...)

In [43]:
state_df.head()

,state,country,date,confirmed_count,lat,long,new_cases
599,DE,US,2020-03-21,45,39.428900,-75.587250,7.0
1319,LA,US,2020-03-21,585,30.256833,-90.683800,47.0
3119,VI,US,2020-03-21,3,18.335800,-64.896300,0.0
299,CA,US,2020-03-21,1364,37.209448,-120.860122,187.0
2099,NJ,US,2020-03-21,1327,40.483722,-74.404933,437.0
